In [ ]:
import pandas as pd
import nglview
from bioexplorer import BioExplorer, Protein, ParametricMembrane, Cell, Volume, Vector3, Quaternion
%matplotlib notebook

In [ ]:
# url = 'r1i5n33.bbp.epfl.ch:5000'
url = 'localhost:5000'

be = BioExplorer(url)
be.reset()

In [ ]:
simulation_id = 3

generate_snapshot = False
generate_movie = False

be.set_general_settings(model_visibility_on_creation=False)

representation=be.REPRESENTATION_ATOMS_AND_STICKS
membrane_size = 800.0
membrane_height = 80

In [ ]:
# Resources
# resource_folder = '../../tests/test_files/'
resource_folder = '/home/favreau/bluebrain/BioExplorer/bioexplorer/pythonsdk/tests/test_files/'
pdb_folder = resource_folder + 'pdb/'
membrane_folder = pdb_folder + 'membrane/'
ion_channels_folder = pdb_folder + 'ion_channels/'
metabolites_folder = pdb_folder + 'metabolites/'

glucose_path = pdb_folder + 'glucose.pdb'

## Metabolites

In [ ]:
import psycopg2
import pandas as pd

db_host = 'localhost'
db_name = 'bioexplorer'
db_user = 'brayns'
db_password ='brayns'
db_schema = 'metabolism'

if True:
    db_host = 'bbpdbsrv01.bbp.epfl.ch'
    db_name = 'bioexplorer'
    db_user = 'bioexplorer'
    db_password = 'verole@2020'
    db_schema = 'metabolism'


db_connection_string = 'host=' + db_host + ' port=5432 dbname=' + db_name + \
              ' user=' + db_user + ' password=' + db_password

db_connection=psycopg2.connect(db_connection_string)
print('Connected to ' + db_name + '.' + db_schema)


In [ ]:
# Variables
def get_variables():
    variables = dict()
    sql_command = "SELECT guid, pdb_guid FROM %s.variable" % db_schema
    data = pd.read_sql(sql_command, db_connection)
    for i in range(len(data)):
        variables[str(data['guid'][i])] = data['pdb_guid'][i]
    return variables

def get_concentration(variable_guid, simulation_id, frame):
    sql_command = 'SELECT v.location_guid AS guid, c.concentration / v.base_value AS concentration FROM %s.variable as v, %s.concentration AS c WHERE c.variable_guid=%d AND v.guid=c.variable_guid AND c.timestamp=%d AND c.simulation_guid=%d' % (db_schema, db_schema, variable_guid, frame, simulation_id)
    data = pd.read_sql(sql_command, db_connection)
    if(len(data)>0):
        return(float(data['concentration']))
    return 0.0


get_concentration(0, 3, 0)

In [ ]:
import glob
import random
import os

variables = get_variables()
simulation_guid = 3
frame = 0

count = 0
for variable in variables:
    pdb_guid = variables[variable]
    if not pdb_guid:
        continue
    variable_guid = int(variable)
    file_name = metabolites_folder + pdb_guid + '.pdb'
    concentration = get_concentration(variable_guid, simulation_guid, frame)
    nb_molecules = int(100000 * concentration)
    print('%d: %d' % (variable_guid, nb_molecules))
    try:
        metabolite = Protein(
            sources=[file_name], load_bonds=True, load_hydrogen=True,
            load_non_polymer_chemicals=True, 
            occurences=nb_molecules)
        volume = Volume(
            name=str(variable_guid), size=800, protein=metabolite,
            random_position_seed=count, random_position_stength=membrane_size / 600.0,
            random_rotation_seed=count + 1, random_rotation_stength=0.3
        )
        status = be.add_volume(
            volume=volume, representation=representation,
            random_seed=count)
    except Exception as e:
        print(e)
    count += 2


In [19]:
variables

{'98': '409682',
 '11': '5957',
 '38': '439153',
 '36': '5892',
 '164': '',
 '139': '',
 '32': '5957',
 '165': '',
 '37': '51',
 '149': '',
 '160': '',
 '158': '122357',
 '152': '',
 '143': '439284',
 '89': '',
 '106': '',
 '67': '8468',
 '6': '91826668',
 '7': '729',
 '8': '',
 '18': '',
 '17': '',
 '13': '65533',
 '14': '5793',
 '12': '750',
 '41': '444972',
 '30': '729',
 '40': '',
 '26': '',
 '24': '',
 '35': '33032',
 '52': '440641',
 '50': '33032',
 '51': '',
 '56': '',
 '44': '6022',
 '55': '',
 '49': '1712093',
 '45': '5957',
 '48': '51',
 '43': '',
 '54': '',
 '47': '5960',
 '57': '',
 '53': '',
 '46': '6083',
 '70': '',
 '142': '6083',
 '144': '91826668',
 '145': '5886',
 '16': '',
 '163': None,
 '96': '',
 '95': '',
 '97': '2879',
 '90': '6133',
 '99': '33032',
 '101': '122357',
 '103': '',
 '100': '119',
 '110': '124886',
 '105': '1005',
 '126': '439153',
 '102': '',
 '109': '',
 '23': '',
 '66': '977',
 '133': '977',
 '20': '977',
 '9': '6022',
 '129': '',
 '85': '',
 '84'

In [ ]:
be.set_models_visibility(True)

## Neuron

In [ ]:
name = 'Neuron'

# Ion Channel
ion_channel = Protein(
    sources=[ion_channels_folder + '6uz3.pdb'],
    occurences=50, position=Vector3(0.0, 5.0, 0.0),
    rotation=Quaternion(0, 0.707, 0.707, 0),
    load_non_polymer_chemicals=False,
)

# Membrane definition
membrane = ParametricMembrane(
    sources=[
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ],
    occurences=400000
)

# Cell definition
neuron = Cell(
    name=name,
    size=membrane_size,
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    extra_parameters=[membrane_height],
    membrane=membrane,
    random_position_seed=1, random_position_strength=0.025,
    random_rotation_seed=2, random_rotation_strength=0.5,
    receptor=ion_channel)

# Add cell to scene
status = be.add_cell(
    cell=neuron, representation=representation,
    random_seed=1)

In [ ]:
name = 'NeuronMitochondrion'

# Ion Channel
ion_channel = Protein(
    sources=[ion_channels_folder + '6uz3.pdb'],
    occurences=0, position=Vector3(0.0, 5.0, 0.0),
    rotation=Quaternion(0, 0.707, 0.707, 0),
    load_non_polymer_chemicals=False,
)

# Membrane definition
membrane = ParametricMembrane(
    sources=[
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ],
    occurences=400000
)

# Cell definition
neuron_mtochodrion = Cell(
    name=name,
    size=membrane_size,
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    extra_parameters=[membrane_height],
    membrane=membrane,
    random_position_seed=1, random_position_strength=0.025,
    random_rotation_seed=2, random_rotation_strength=0.5,
    receptor=ion_channel)

# Add cell to scene
status = be.add_cell(
    cell=neuron_mtochodrion, representation=representation,
    position=Vector3(0, -20, 0), random_seed=2)

## Astrocyte

In [ ]:
name = 'Astrocyte'
representation=be.REPRESENTATION_ATOMS_AND_STICKS

# Ion Channel
ion_channel = Protein(
    sources=[ion_channels_folder + '6uz3.pdb'],
    occurences=50, position=Vector3(0.0, 5.0, 0.0),
    rotation=Quaternion(0, 0.707, 0.707, 0),
    load_non_polymer_chemicals=False,
)

# Membrane definition
membrane = ParametricMembrane(
    sources=[
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ],
    occurences=400000)

# Cell definition
astrocyte = Cell(
    name=name,
    size=membrane_size,
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    extra_parameters=[membrane_height],
    membrane=membrane,
    random_position_seed=1, random_position_strength=0.025,
    random_rotation_seed=2, random_rotation_strength=0.5,
    receptor=ion_channel)

# Add cell to scene
status = be.add_cell(
    cell=astrocyte, representation=representation,
    rotation=Quaternion(0.0, 1.0, 0.0, 0.0), position=Vector3(0, 100, 0),
    random_seed=3)

In [ ]:
name = 'AstrocyteMitochondrion'

# Ion Channel
ion_channel = Protein(
    sources=[ion_channels_folder + '6uz3.pdb'],
    occurences=0, position=Vector3(0.0, 5.0, 0.0),
    rotation=Quaternion(0, 0.707, 0.707, 0),
    load_non_polymer_chemicals=False,
)

# Membrane definition
membrane = ParametricMembrane(
    sources=[
        membrane_folder + 'segA.pdb',
        membrane_folder + 'segB.pdb',
        membrane_folder + 'segC.pdb',
        membrane_folder + 'segD.pdb'
    ],
    occurences=400000
)

# Cell definition
neuron_mtochodrion = Cell(
    name=name,
    size=membrane_size,
    shape=be.ASSEMBLY_SHAPE_SINUSOIDAL,
    extra_parameters=[membrane_height],
    membrane=membrane,
    random_position_seed=1, random_position_strength=0.025,
    random_rotation_seed=2, random_rotation_strength=0.5,
    receptor=ion_channel)

# Add cell to scene
status = be.add_cell(
    cell=neuron_mtochodrion, representation=representation,
    rotation=Quaternion(0.0, 1.0, 0.0, 0.0), position=Vector3(0, 120, 0),
    random_seed=4)

## Materials

In [ ]:
COLOR_MAPS = [
    'Accent', 'Accent_r', 'Blues', 'Blues_r', 'BrBG', 'BrBG_r', 'BuGn', 'BuGn_r',
    'BuPu', 'BuPu_r', 'CMRmap', 'CMRmap_r', 'Dark2', 'Dark2_r', 'GnBu', 'GnBu_r',
    'Greens', 'Greens_r', 'Greys', 'Greys_r', 'OrRd', 'OrRd_r', 'Oranges', 'Oranges_r',
    'PRGn', 'PRGn_r', 'Paired', 'Paired_r', 'Pastel1', 'Pastel1_r', 'Pastel2', 'Pastel2_r',
    'PiYG', 'PiYG_r', 'PuBu', 'PuBuGn', 'PuBuGn_r', 'PuBu_r', 'PuOr', 'PuOr_r',
    'PuRd', 'PuRd_r', 'Purples', 'Purples_r', 'RdBu', 'RdBu_r', 'RdGy', 'RdGy_r',
    'RdPu', 'RdPu_r', 'RdYlBu', 'RdYlBu_r', 'RdYlGn', 'RdYlGn_r', 'Reds', 'Reds_r',
    'Set1', 'Set1_r', 'Set2', 'Set2_r', 'Set3', 'Set3_r',
    'Wistia', 'Wistia_r', 'YlGn', 'YlGnBu', 'YlGnBu_r', 'YlGn_r', 'YlOrBr',
    'YlOrBr_r', 'YlOrRd', 'YlOrRd_r', 'afmhot', 'afmhot_r', 'autumn', 'autumn_r',
    'binary', 'binary_r', 'bone', 'bone_r', 'brg', 'brg_r', 'bwr', 'bwr_r', 'cool', 'cool_r',
    'coolwarm', 'coolwarm_r', 'copper', 'copper_r', 'cubehelix', 'cubehelix_r', 'flag',
    'flag_r', 'gist_earth', 'gist_earth_r', 'gist_gray', 'gist_gray_r', 'gist_heat',
    'gist_heat_r', 'gist_ncar', 'gist_ncar_r', 'gist_rainbow', 'gist_rainbow_r', 'gist_stern',
    'gist_stern_r', 'gist_yarg', 'gist_yarg_r', 'gnuplot', 'gnuplot2', 'gnuplot2_r',
    'gnuplot_r', 'gray', 'gray_r', 'hot', 'hot_r', 'hsv', 'hsv_r', 'icefire', 'icefire_r',
    'inferno', 'inferno_r', 'jet_r', 'magma', 'magma_r', 'mako', 'mako_r',
    'nipy_spectral', 'nipy_spectral_r', 'ocean', 'ocean_r', 'pink', 'pink_r', 'plasma',
    'plasma_r', 'prism', 'prism_r', 'rainbow', 'rainbow_r', 'rocket', 'rocket_r', 'seismic',
    'seismic_r', 'spring', 'spring_r', 'summer', 'summer_r',
    'tab10', 'tab10_r', 'tab20', 'tab20_r', 'tab20b', 'tab20b_r', 'tab20c', 'tab20c_r',
    'terrain', 'terrain_r', 'viridis', 'viridis_r', 'vlag', 'vlag_r', 'winter', 'winter_r'
]

def set_color_scheme(
        shading_mode, user_parameter=3.0, specular_exponent=5.0, glossiness=1.0):
    """
    Apply a default color scheme to all components in the scene

    :shading_mode: Shading mode (None, basic, diffuse, electron, etc)
    :user_parameter: User parameter specific to each shading mode
    :specular_exponent: Specular exponent for diffuse shading modes
    :glossiness: Glossiness
    """

    import seaborn as sns
    model_ids = be.get_model_ids()
    global_palette = sns.color_palette('rainbow', len(model_ids["ids"]))

    index = 0
    for model_id in model_ids["ids"]:
        model_name = be.get_model_name(model_id)['name']
        material_ids = be.get_material_ids(model_id)["ids"]
        nb_materials = len(material_ids)

        if model_name.find('Neuron') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Blues", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        elif model_name.find('Astrocyte') != -1 and model_name.find('Mitochondrion') == -1:
            palette = sns.color_palette("Wistia", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        elif model_name.find('Mitochondrion') != -1:
            palette = sns.color_palette("Purples_r", nb_materials)
            be.set_materials_from_palette(
                model_ids=[model_id],
                material_ids=material_ids,
                palette=palette,
                shading_mode=shading_mode,
                user_parameter=user_parameter,
                glossiness=glossiness,
                specular_exponent=specular_exponent,
            )
        else:
            material_ids = be.get_material_ids(model_id)['ids']
            colors = list()
            shading_modes = list()
            user_parameters = list()
            glossinesses = list()
            specular_exponents = list()

            for m in material_ids:
                colors.append(global_palette[index])
                shading_modes.append(shading_mode)
                user_parameters.append(user_parameter)
                glossinesses.append(glossiness)
                specular_exponents.append(specular_exponent)

            be.set_materials(
                model_ids=[model_id],
                material_ids=material_ids,
                diffuse_colors=colors,
                specular_colors=colors,
                shading_modes=shading_modes,
                user_parameters=user_parameters,
                glossinesses=glossinesses,
                specular_exponents=specular_exponents
            )
        index += 1


set_color_scheme(shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50)

In [ ]:
be.set_models_visibility(True)

## Snapshots

In [ ]:
be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)

In [ ]:
from bioexplorer import MovieMaker
mm = MovieMaker(be)

In [ ]:
be.core_api().set_camera(
    orientation = [0.001082250237181038, 0.0010822502371810592, -1.1712669477445858e-06, 0.9999988287330522],
    position = [-52.6789638487797, 39.07960017636351, 527.9410106263259],
    target = [-53.924311289991905, 40.32495053485112, -47.408608694324016]
)

In [ ]:
if True: # generate_snapshot:
    mm.create_snapshot(
        base_name = 'Metabolism_Molecular_Level_4k_v8',
        path='/gpfs/bbp.cscs.ch/project/proj129/images/metabolism/4K',
        size=[3840, 2160], samples_per_pixel=64
)

In [ ]:
be.core_api().set_camera(
    orientation = [0.7071067811865472, -0.7071067811865478, -1.4977640110329394e-17, 1.0157324573387875e-16],
    position = [-3.4611744697862217, 57.47436388665844, -508.2669299940219],
    target = [-3.4611744697861457, 57.474363886658544, -47.548320280825195]    
)

In [ ]:
if True: # generate_snapshot:
    mm.create_snapshot(
        base_name = 'Metabolism_Molecular_Level_8k_v5',
        path='/gpfs/bbp.cscs.ch/project/proj129/images/metabolism/4K',
        size=[2 * 3840, 2 * 2160], samples_per_pixel=64
    )

## Widgets

In [ ]:
from bioexplorer import Widgets
w = Widgets(be)

In [ ]:
w.display_focal_distance()